In [1]:
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta



source = pd.read_csv('Consumption.csv')

#seperate 2020 data due to different date encoding

data_2020 = source.loc[source['Date'].str.startswith('2020')]

data_rest = source.loc[~source['Date'].str.startswith('2020')]

#2020 convert to dateitime

data_2020['Date'] = pd.to_datetime(data_2020['Date'],format=r'%Y%m%d')
data_2020.sort_values(by='Date',ascending=True)

data_rest['Date'] = pd.to_datetime(data_rest['Date'],dayfirst=True)
data_rest.sort_values(by='Date',ascending=True)

merged_source = data_2020.append(data_rest)
merged_source.sort_values(by='Date',ascending=True)


def return_mm_dd(date):

    return(str(date.month))

merged_source['mm-dd'] = merged_source['Date'].apply(lambda x: dt.date.strftime(x,r"%m-%d"))
merged_source['y'] = merged_source['Date'].apply(lambda x: dt.date.strftime(x,r"%Y"))

pre_pivot = merged_source[['Consumption','mm-dd','y']]

pivot = pre_pivot.pivot_table(index='mm-dd',columns='y',values='Consumption')

def get_max_min(row):
    row.to_list()

columns_2016_to_2020 = ['2016','2017','2018','2019','2020']


pivot['Max'] = pivot.apply(lambda x: max(x[columns_2016_to_2020].to_list()),axis=1)
pivot['Min'] = pivot.apply(lambda x: min(x[columns_2016_to_2020].to_list()),axis=1)
pivot['Avg'] = pivot.apply(lambda x: np.nanmean(x[columns_2016_to_2020].to_list()),axis=1)

pivot['Month'] = pivot.index.to_series().apply(lambda x: x[:2])#returns month no.

#further pivots



range_pivot = pd.pivot_table(pivot, values=['Min','Max','Avg','2021','2022'], index=['Month'],
                    aggfunc={'Min': np.min,
                             'Max': np.max,
                             'Avg': np.nanmean,
                             '2021':np.nanmean,
                             '2022':np.nanmean})




# Set up Figure
p1_fig = go.Figure()





# Create x-axis labels for the months
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# Create the Graphs
#pivot.to_csv('pivot.csv')

data = range_pivot

p1_fig.add_trace(go.Scatter(x=months, y=data['Max'],
    fill=None,
    mode=None,
    line_color='lightgray', showlegend=False
    ))

p1_fig.add_trace(go.Scatter(x=months, y=data['Min'],
    fill='tonexty', # fill area between trace0 and trace1
    mode=None, line_color='lightgray', showlegend=False))


p1_fig.add_trace(go.Scatter(x=months, y=data['Avg'], name='2016-2020 Avg',
                                 line=dict(color='black', width=4, dash='dot')))

p1_fig.add_trace(go.Scatter(x=months, y=data['2021'], name='2021',
                                 line=dict(color='darkblue', width=4)))
p1_fig.add_trace(go.Scatter(x=months, y=data['2022'], name='2022',
                                 line=dict(color='firebrick', width=4)))

p1_fig.show()


C:\Users\sbuff\AppData\Local\Temp\ipykernel_19840\1579003133.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020['Date'] = pd.to_datetime(data_2020['Date'],format=r'%Y%m%d')
C:\Users\sbuff\AppData\Local\Temp\ipykernel_19840\1579003133.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rest['Date'] = pd.to_datetime(data_rest['Date'],dayfirst=True)
C:\Users\sbuff\AppData\Local\Temp\ipykernel_19840\1579003133.py:27: FutureWarning: The frame.append method is deprecated and will be removed from

Here we can see the trends in an easy to digest manner, the range between 2016 - 2020 shows historical trends indicating the range of consumption. we can see that 2022 compared to the 2016 to 2020 average and 2021's consumption started lower at the start of the year, and continued to lag behind throughout all the way up to September, based on previous datapoints throughout the year for 2022 I predict consumption to continue to lag behind 2021. This could be due to the ongoing conflict in Ukraine affecting the price and thus consumption.